In [2]:
import numpy as np
import pickle as pkl
from Extractor import Extractor
from scipy.sparse import coo_matrix,csr_matrix
#import tensorflow as tf
import torch
from utils import *
import networkx as nx
from matplotlib import pyplot as plt
from model import *
from train import *

from explain import *
import time
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import to_networkx

import sys
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device in use: ', device)

np.set_printoptions(threshold=sys.maxsize)


device in use:  cpu


In [3]:
class BA_Shape_Dataset(Dataset):
    def __init__(self, root, name, setting=1, hops=3, transform = None, pre_transform = None, subgraph = False, remap=False):
        super(BA_Shape_Dataset, self).__init__(root, transform, pre_transform)
        self.root = root
        self.subgraph = subgraph
        self.remap = remap
        self.name = name
        self.setting =setting
        with open(os.path.join(self.root, name + '.pkl'), 'rb') as fin:
            self.adj, self.features, self.y_train, self.y_val, self.y_test, self.train_mask, self.val_mask, self.test_mask, self.edge_label_matrix  = pkl.load(fin)
        self.hops = hops

        self.all_label = np.logical_or(self.y_train,np.logical_or(self.y_val,self.y_test))
        self.single_label = np.argmax(self.all_label,axis=-1)
        #self.allnodes = [i for i in range(self.single_label.shape[0]) if self.single_label[i] != 0] #[i for i in range(400,700,5)]
        self.csr_adj = csr_matrix(self.adj)
        self.extractor = Extractor(self.csr_adj,self.features,self.edge_label_matrix,self.all_label,self.hops)            
    @property
    def num_features(self):
        return 10
    @property
    def num_classes(self):
        return self.all_label.shape[1]

    @property 
    def allnodes(self):
        if self.setting==1:
            if self.name=='syn3':
                allnodes = [i for i in range(511,871,6)]
            elif self.name=='syn4':
                allnodes = [i for i in range(511,800,1)]
            else:
                allnodes = [i for i in range(400,700,5)] # setting from their original paper
        elif self.setting==2:
            allnodes = [i for i in range(self.single_label.shape[0]) if self.single_label[i] ==1]
        elif self.setting==3:
            if self.name == 'syn2':
                allnodes = [i for i in range(self.single_label.shape[0]) if self.single_label[i] != 0 and self.single_label[i] != 4]
            else:
                allnodes = [i for i in range(self.single_label.shape[0]) if self.single_label[i] != 0]
        return allnodes

    def set_hops(self, hops=3):
        self.hops = hops
        self.extractor = Extractor(self.csr_adj,self.features,self.edge_label_matrix,self.all_label,self.hops)  

    def get_subgraph(self, idx):
        #if self.subgraph:
        idx = self.allnodes[idx] if self.remap else idx
        sub_adj,sub_feature, sub_label,sub_edge_label_matrix = dataset.extractor.subgraph(idx)
        return sub_adj,sub_feature, sub_label,sub_edge_label_matrix
        #else:
        #    return None
    def len(self):
        if self.subgraph:
            if not self.remap:
                return len(self.single_label)
            return len(self.allnodes)
        else:
            return 1
    
    def get(self, idx):
        if self.subgraph:
            if self.remap:
                idx = self.allnodes[idx]
            sub_adj,sub_feature, sub_label,sub_edge_label_matrix = self.extractor.subgraph(idx)
            edge_index = torch.tensor(preprocess_adj(sub_adj)[0].T, dtype = torch.long)
            x = torch.tensor(sub_feature).float()
            y = torch.argmax(torch.tensor(sub_label, dtype = torch.int32), dim=-1)
            data = Data(edge_index = edge_index, x = x, y = y)
        else:

            edge_index = torch.tensor(preprocess_adj(self.adj)[0].T, dtype = torch.long)
            x = torch.tensor(self.features).float()
            y = torch.argmax(torch.tensor(np.logical_or(self.y_train,np.logical_or(self.y_val,self.y_test)), dtype = torch.int32), dim=-1)
            data = Data(edge_index = edge_index, x = x, y = y)
        return data

In [4]:
dataset = BA_Shape_Dataset(root = './dataset', name = 'syn2', setting = 3)

explainer


In [16]:
gcn_model = GCN_Node(dataset, 3, 20)
dataset.set_hops(3)

explainer


In [5]:
gcn_model = torch.load('./checkpoint/gcn_community_', map_location=torch.device(device))

In [6]:
acc_list = []
auc_list = []

dataset.subgraph = False
dataset.remap = False
dataset.setting=3
load_model = gcn_model

load_model.eval()

all_node_label = []
all_node_color = []
all_elapsed = []
all_len = []
for idx in dataset.allnodes:
    #idx = 313
    print('\n===========index: ', idx, '============')
    sub_adj,sub_feature, sub_label,sub_edge_label_matrix = dataset.get_subgraph(idx)
    #truth_node = np.where(sub_label[:,1] == True)[0]
    truth_node = list(get_node_set(sub_edge_label_matrix))
    
    class_idx = np.argmax(sub_label[0],axis=-1)
    print('0 label: ', class_idx)
    node_range = dataset.extractor.nodes
    #print('node range: ', node_range)
    node_sort, node_color, elapsed = print_subgraph_explain(dataset = dataset, model = load_model, idx = 0, class_idx = class_idx, visible = False, figsize = (12,9), node_range = node_range)
    #print(node_sort)
    all_elapsed.append(elapsed)
    all_len.append(len(node_color))
    node_label = np.array([0] * sub_label.shape[0])
    #node_label[list(truth_node)] = 1
    # find truth node, far node is not real truth
    for n in truth_node:
        if abs((node_range[n] - node_range[0])) <= 8:
            node_label[n] = 1
            #print(n)

    try:
        auc = roc_auc_score(node_label, node_color)
    except:
        print('foo')
        continue
        #auc = 1.0

    #print("truth node: ", truth_node)
    #print(node_sort)
    acc = len([node for node in node_sort[:5] if node in truth_node])/5
    #all_node_color.extend(node_color)
    print('acc: ', acc)
    print('auc: ', auc)
    acc_list.append(acc)
    auc_list.append(auc)
    #if acc == 0.0:
    #    print(node_sort)
    #    print_explain(dataset, load_model, idx, class_idx = np.argmax(sub_label[0],axis=-1), visible = True)
    print('mean acc: ', np.mean(acc_list))
    print('mean auc: ', np.mean(auc_list))


===========index:  300 ============
0 label:  1
target node:  300
epoch time:  4.825165271759033
acc:  1.0
auc:  1.0
mean acc:  1.0
mean auc:  1.0

===========index:  301 ============
0 label:  1
target node:  301
epoch time:  1.8858258724212646
acc:  1.0
auc:  1.0
mean acc:  1.0
mean auc:  1.0

===========index:  302 ============
0 label:  2
target node:  302
epoch time:  0.2862284183502197
acc:  1.0
auc:  1.0
mean acc:  1.0
mean auc:  1.0

===========index:  303 ============
0 label:  2
target node:  303
epoch time:  1.449007272720337
acc:  1.0
auc:  1.0
mean acc:  1.0
mean auc:  1.0

===========index:  304 ============
0 label:  3
target node:  304
epoch time:  0.5323390960693359
acc:  1.0
auc:  1.0
mean acc:  1.0
mean auc:  1.0

===========index:  305 ============
0 label:  1
target node:  305
epoch time:  2.7955689430236816
acc:  0.8
auc:  0.9377510040160643
mean acc:  0.9666666666666667
mean auc:  0.9896251673360107

===========index:  306 ============
0 label:  1
target node:  